# NBA.com stats crawler
Using selenium and BeautifulSoup with Python 3.x
> LATIA (2023, Fall) TA: [王修佑](https://github.com/whyhugo)

In [1]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from lxml import etree
import time

In [2]:
'''不想彈出視窗可以用下面這段 code
chrome_options = Options()
chrome_options.add_argument('--headless')
, options=chrome_options
'''

url = 'https://www.nba.com/stats/players/traditional'
driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')
html = driver.get(url)
sp = soup(driver.page_source, 'html.parser')

C:\Users\user\AppData\Local\Temp\ipykernel_12324\2747250977.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


### Get Column Name Using find & find_all

In [3]:
def get_column_name():
    data_column_name = []
    sp = soup(driver.page_source, 'html.parser')
    th_tags = sp.find('tr', class_='Crom_headers__mzI_m').find_all('th')
    for th_tag in th_tags:
        data_column_name.append(th_tag.text)
    data_column_name = data_column_name[1:30]
    return data_column_name

### Click Next Page Button Using XPath

In [4]:
# click next page button
def click_next_page_button():
    time.sleep(3)
    element = driver.find_element('xpath', '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[5]/button[2]')
    driver.execute_script("arguments[0].click();", element)
    time.sleep(2)

### Get Row Data in the Table Using find, find_all & Nested Loop

In [5]:
def get_data():
    data = []
    row = []
    count = 0
    sp = soup(driver.page_source, 'html.parser')
    tr_tags = sp.find('tbody', class_='Crom_body__UYOcU').find_all('tr')
    for tr_tag in tr_tags:
        td_tags = tr_tag.find_all('td')
        for td_tag in td_tags:
            row.append(td_tag.text)
            count += 1
            if count % 30 == 0:
                data.append(row[1:])
                row = []
    return data

In [ ]:
try:
    df_list = []
    sp = soup(driver.page_source, 'html.parser')
    total_row = sp.find('div', class_='Pagination_content__f2at7 Crom_cromSetting__Tqtiq').find('div').text[:-5]
    while True:
        print('New Page!')
        time.sleep(2)
        df_list.extend(get_data())
        if len(df_list) == int(total_row):
            print('This is the last page ( •̀ ω •́ )✧')
            break
        else:
            click_next_page_button()
except:
    print('Something went wrong. QAQ')

In [7]:
df = pd.DataFrame(df_list, columns=get_column_name())
df

,Player,Team,Age,GP,W,L,Min,PTS,FGM,FGA,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
0,Joel Embiid,PHI,29,66,43,23,34.6,33.1,11.0,20.1,...,10.2,4.2,3.4,1.0,1.7,3.1,56.2,39.0,1.0,6.4
1,Luka Doncic,DAL,24,66,33,33,36.2,32.4,10.9,22.0,...,8.6,8.0,3.6,1.4,0.5,2.5,56.8,36.0,10.0,1.9
2,Damian Lillard,POR,32,58,27,31,36.3,32.2,9.6,20.7,...,4.8,7.3,3.3,0.9,0.3,1.9,49.1,16.0,2.0,1.8
3,Shai Gilgeous-Alexander,OKC,24,68,33,35,35.5,31.4,10.4,20.3,...,4.8,5.5,2.8,1.6,1.0,2.8,50.4,3.0,0.0,2.2
4,Giannis Antetokounmpo,MIL,28,63,47,16,32.1,31.1,11.2,20.3,...,11.8,5.7,3.9,0.8,0.8,3.1,54.8,46.0,6.0,5.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,Alondes Williams,BKN,24,1,1,0,5.3,0.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,1.0,-0.8,0.0,0.0,-5.0
535,Deonte Burton,SAC,29,2,1,1,3.2,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5
536,Frank Jackson,UTA,25,1,0,1,5.1,0.0,0.0,3.0,...,2.0,1.0,0.0,0.0,0.0,0.0,3.9,0.0,0.0,-2.0
537,Michael Foster Jr.,PHI,20,1,1,0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0


In [8]:
df.to_csv('nba_stats.csv', index=False, encoding='utf-8')

In [ ]:
driver.quit()